In [1]:
import pandas as pd
import glob
from difflib import SequenceMatcher
import re

# Helper functions

In [19]:
def addToPrev(prev, index, text):
    prev[index] = text
    return prev

################################

def inPrev(prev, text, tot_sims):
    for message in prev:
        if SequenceMatcher(None, text, message).ratio() >= 0.7:
            tot_sims.setdefault(message, []).append(text)
            return True, tot_sims
    return False, tot_sims

################################

def reduce_tweets(tweets, tweets_ids, reduceBy): 
    prev = [""]*reduceBy
    index = 0
    tot_sims = {}
    filtered_tweets = []
    filtered_tweets_ids = []
    
    for i in range(len(tweets)):
        in_prev, tot_sims = inPrev(prev, tweets[i], tot_sims)
        if not in_prev:
            prev = addToPrev(prev, index, tweets[i])
            index = (index + 1) % reduceBy
            filtered_tweets.append(tweets[i])
            filtered_tweets_ids.append(tweets_ids[i])
    
    return filtered_tweets, filtered_tweets_ids

################################

def check_sim(check_word, sims):
    words = ["mosquito", "protect", "repel", "spray"]
    for word in words:
        ratio = SequenceMatcher(None, check_word, word).ratio()
        if ratio >= 0.7:
            sims[check_word] = ratio
            return True, sims
    return False, sims

################################

def check_sim_v2(check_word, sims):
    if re.compile('m[a-z]+sq[a-z]+to').match(check_word):
        sims.append(check_word)
        return True, sims
    
    for word in ['repel', 'protect', 'spray']:
        if re.compile(word).search(check_word):
            sims.append(check_word)
            return True, sims
    
    return False, sims

# Store Tweets

In [3]:
path = "/Users/datacsv/*.csv"

In [5]:
df = pd.read_csv("/Users/datacsv/tweets_2015_08_20.csv")

In [6]:
test = df[df["Language"] == "en"][['Tweet_ID', 'Message']]
test.head()

,Tweet_ID,Message
4,634152976479862788,MEU DEUS AFASTA ESSA ZICA
10,634153565460652032,ESSA ZICA EU SABIA QUE IA ACABAR HOJE
48,634155015901454336,@rodrigo_schndr &gt; Cover me! &gt; Roger Th...
65,634155949603835905,ZICA
87,634156157372862465,SAI ZICA PORRA


In [7]:
tweets = test["Message"]
ids = test["Tweet_ID"]

In [13]:
test.iloc[87]

Tweet_ID                     634185437964279808
Message     RT @sccpdiniz: SAI ZICA DO CARALHO!
Name: 1219, dtype: object

In [14]:
tweets = []
tweets_ids = []
regex = re.compile('[^a-zA-Z]')
sims = []
for file in glob.glob(path):
    df = pd.read_csv(file, sep=',', usecols=['Tweet_ID', 'Message', 'Language'])
    df_tweets = df[df["Language"] == "en"][['Tweet_ID', 'Message']]
    
    for i in df_tweets.index:
        tweet = df_tweets.loc[i]["Message"]
        if tweet[0:2] != "RT":
            for word in tweet.lower().split():
                isSimilar, sims = check_sim_v2(regex.sub('', word), sims)
                if(isSimilar):
                    tweets.append(tweet)
                    tweets_ids.append(df_tweets.loc[i]["Tweet_ID"])
                    break
                    
                    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
print("Number of tweets that contain mosquito or a similar word is: {}".format(len(tweets)))
#357052

Number of tweets that contain mosquito or a similar word is: 454661


In [16]:
pd.DataFrame({"tweet_id": tweets_ids, "tweet": tweets}).to_csv("all_tweets.csv")

### Remove redundant tweets

In [20]:
filtered_tweets, filtered_tweets_ids = reduce_tweets(tweets, tweets_ids, 10)
print("Number of tweets after prev 10 algorithm: {}".format(len(filtered_tweets)))


Number of tweets after prev 10 algorithm: 260654


In [21]:
pd.DataFrame({"filtered_tweets_ids": filtered_tweets_ids, "filtered_tweets": filtered_tweets}).to_csv("filtered_tweets.csv")


### Sims

In [22]:
pd.DataFrame({"sims": list(set(sims))}).to_csv("sims.csv")